In [1]:
import torch
from torch import nn
from sklearn import model_selection
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from IPython import display
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

import typing
%matplotlib inline

In [2]:
com = pd.read_csv('Top_100_Companies_Load_Balancing_Data.csv')
print(len(com))
#sn.pairplot(diabetes_db)

100


In [3]:
def clean_training_data(df):
    # 1. Remove duplicate rows
    df = df.drop_duplicates()

    # 2. Drop rows with too many missing values (threshold: 50% missing)
    # df = df.dropna(thresh=df.shape[1] * 0.5)

    # 3. Fill remaining missing values with column mean (for numerical data)
    for col in df.select_dtypes(include=['number']).columns:
        df[col].fillna(df[col].mean(), inplace=True)

    # 4. Remove outliers using the IQR method
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

    # 5. Normalize numerical features using StandardScaler
    # scaler = StandardScaler()
    # numeric_cols = df.select_dtypes(include=['number']).columns
    # df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    # Лексикографическое полуупорядочивание
    df = df.sort_values(by=["Error Rate (%)", "Avg Latency (ms)", "Throughput (Gbps)"], ascending=[True, True, False], kind='stable')
    # Отрезаем 20% последних данных
    df = df.iloc[:round(len(df) * 0.5)]

    return df

In [4]:
for col in com.select_dtypes(include=['object']).columns:
    # factorize returns a tuple (labels, uniques); we only need the labels.
    com[col] = pd.factorize(com[col])[0]
missing_counts = com.isna().sum()
print(missing_counts)
com = clean_training_data(com)
com.fillna(com.mean(numeric_only=True), inplace=True)
com.head(2)

com.drop(columns=['Company'])

Company                       0
Load Balancing Method         0
Users                         0
Servers                       0
Avg Latency (ms)              0
Requests per Second (RPS)     0
Peak RPS                      0
Error Rate (%)                0
Throughput (Gbps)             0
Daily Active Users (DAU)      0
Monthly Active Users (MAU)    0
Avg Session Time (min)        0
Cloud Provider                0
Failover Mechanism            0
dtype: int64


,Load Balancing Method,Users,Servers,Avg Latency (ms),Requests per Second (RPS),Peak RPS,Error Rate (%),Throughput (Gbps),Daily Active Users (DAU),Monthly Active Users (MAU),Avg Session Time (min),Cloud Provider,Failover Mechanism
25,5,1280640554,36610,24.17,2729373,11273243.71,0.04,59.88,629806899,693554301,54.13,2,0
12,3,295108266,51149,129.96,3802218,14826755.56,0.10,371.76,123398117,141956183,42.47,3,2
22,3,2264382111,53874,149.21,1541915,2529604.63,0.14,63.52,503425490,933920521,11.99,2,2
92,9,1680638659,12580,98.69,2884495,8558186.74,0.24,451.33,659161299,1324053195,35.41,4,1
72,0,880081176,48127,133.54,4239027,13407844.56,0.40,293.37,258674965,734094570,20.28,2,0
40,7,2101208734,85902,85.56,4148438,13193582.95,0.42,347.26,793146485,1157269973,24.94,1,1
18,6,192860289,2174,254.19,2368093,3774118.71,0.43,51.20,3416353,71164984,26.72,3,0
86,7,1776030869,91134,225.66,2619494,4693409.92,0.49,304.77,639985710,1205725589,22.33,0,1
2,2,2871416315,75020,139.58,343433,1285326.85,0.50,443.51,49583233,2403018554,53.06,2,2
27,1,1833900920,2786,184.12,1767586,5795560.70,0.50,392.97,613100279,1182407618,14.55,2,0


In [5]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(
    com.loc[:, com.columns != 'Requests per Second (RPS)'],
    com['Requests per Second (RPS)'],
    test_size=0.2,
    random_state=42
    )
x_train = torch.from_numpy(x_train.values).float()
x_test = torch.from_numpy(x_test.values).float()

y_train = torch.from_numpy(y_train.values).float()
y_train = y_train.reshape(-1, 1)

y_test = torch.from_numpy(y_test.values).float()
y_test = y_test.reshape(-1, 1)

print(x_train)

x_train = com.drop(columns=['Requests per Second (RPS)'])
y_train = com['Requests per Second (RPS)']

x_train = torch.from_numpy(x_train.values).float()
y_train = torch.from_numpy(y_train.values).float()
y_train = y_train.reshape(-1, 1)

tensor([[2.1000e+01, 2.0000e+00, 2.7913e+09, 9.6774e+04, 2.2140e+01, 1.8448e+06,
         6.5000e-01, 9.6930e+01, 8.0430e+08, 1.0092e+09, 3.3470e+01, 4.0000e+00,
         1.0000e+00],
        [7.2000e+01, 0.0000e+00, 8.8008e+08, 4.8127e+04, 1.3354e+02, 1.3408e+07,
         4.0000e-01, 2.9337e+02, 2.5867e+08, 7.3409e+08, 2.0280e+01, 2.0000e+00,
         0.0000e+00],
        [1.6000e+01, 2.0000e+00, 1.3097e+09, 9.9472e+04, 2.1650e+02, 1.9764e+06,
         1.8900e+00, 4.3563e+02, 5.1849e+08, 9.6993e+08, 2.7690e+01, 0.0000e+00,
         0.0000e+00],
        [2.0000e+00, 2.0000e+00, 2.8714e+09, 7.5020e+04, 1.3958e+02, 1.2853e+06,
         5.0000e-01, 4.4351e+02, 4.9583e+07, 2.4030e+09, 5.3060e+01, 2.0000e+00,
         2.0000e+00],
        [9.2000e+01, 9.0000e+00, 1.6806e+09, 1.2580e+04, 9.8690e+01, 8.5582e+06,
         2.4000e-01, 4.5133e+02, 6.5916e+08, 1.3241e+09, 3.5410e+01, 4.0000e+00,
         1.0000e+00],
        [1.8000e+01, 6.0000e+00, 1.9286e+08, 2.1740e+03, 2.5419e+02, 3.7741e+06,

In [6]:
gb_regressor = GradientBoostingRegressor(n_estimators=200, learning_rate=0.01, max_depth=10, random_state=42)
gb_regressor.fit(x_train, y_train)

# Делаем предсказание и оцениваем RMSE
preds = gb_regressor.predict(x_test)
# rmse = mean_squared_error(y_test, preds, squared=False)  # RMSE
rmse = mean_squared_error(y_test, preds) ** 0.5  # Используем корень вручную


/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


In [7]:
print(y_test.shape)
preds = torch.tensor(preds)
print(preds.shape)

torch.Size([10, 1])
torch.Size([10])


In [41]:
# y_test = torch.tensor(y_test) if not isinstance(y_test, torch.Tensor) else y_test
# preds = torch.tensor(preds) if not isinstance(preds, torch.Tensor) else preds

# Приведение preds к [20, 1] и объединение
result = torch.cat((y_test, preds.view(-1, 1)), dim=1)

In [42]:
result

tensor([[2729373.0000, 2672135.4152],
        [4765297.0000, 4435286.9797],
        [ 343433.0000,  605862.8806],
        [3802218.0000, 3601240.9909],
        [2149987.0000, 2170375.3631]], dtype=torch.float64)

In [43]:
percantage = abs((y_test.squeeze() - preds)/y_test.squeeze() * 100)
print(percantage)
# avg_per = torch.sum(percantage) / percantage.size(0)
# print(avg_per)

tensor([ 2.0971,  6.9253, 76.4137,  5.2858,  0.9483], dtype=torch.float64)


In [44]:
# Фильтрация выбросов > 50%
filtered_percantage = percantage[percantage <= 50]

# Подсчет среднего без выбросов
if filtered_percantage.size(0) > 0:
    avg_per = torch.sum(filtered_percantage) / filtered_percantage.size(0)
else:
    avg_per = torch.tensor(0.0)  # Защита от деления на ноль

print("\nСреднее процентное отклонение без выбросов:")
print(avg_per)


Среднее процентное отклонение без выбросов:
tensor(3.8141, dtype=torch.float64)


Согласно полученным результатам, обученная модель может определять количество запросов в секунду с погрешнотью в 17%. Испытание было проведено на 80 входных данных для тренировки и 20 для тестирования. Использовался алгоритм обучения Gradient Boosting с фильтрацией входных параметров и преобработкой входных параметров (one-hot encoding, удаление строк со слишком большой разницей мат. ожидания)